# Yash Meshram
# Computer Vision & Internet of Things
# The Sparks Foundation
#
## Task : Detection of face mask
#
## Testing the model on image

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [2]:
prototxtPath = os.path.sep.join([r'/home/yash-meshram/Documents/GitHub/mask-detection/face_detector/', 'deploy.prototxt'])
weightsPath = os.path.sep.join([r'/home/yash-meshram/Documents/GitHub/mask-detection/face_detector', 'res10_300x300_ssd_iter_140000.caffemodel'])

In [3]:
prototxtPath

'/home/yash-meshram/Documents/GitHub/mask-detection/face_detector//deploy.prototxt'

In [4]:
weightsPath

'/home/yash-meshram/Documents/GitHub/mask-detection/face_detector/res10_300x300_ssd_iter_140000.caffemodel'

In [5]:
net = cv2.dnn.readNet(prototxtPath, weightsPath)

In [6]:
net

<dnn_Net 0x7ff108406a50>

In [7]:
model = load_model(r'/home/yash-meshram/Documents/GitHub/mask-detection/mobilenet_v2.model')

In [8]:
#take image and convert it to array---imread()
image = cv2.imread(r'/home/yash-meshram/Documents/GitHub/mask-detection/test_image/example_01.png')

In [9]:
image

array([[[186, 183, 145],
        [186, 184, 144],
        [186, 184, 144],
        ...,
        [223, 220, 215],
        [223, 220, 215],
        [223, 220, 215]],

       [[184, 183, 145],
        [184, 183, 145],
        [184, 183, 145],
        ...,
        [223, 220, 215],
        [223, 220, 215],
        [223, 220, 215]],

       [[182, 181, 146],
        [182, 181, 146],
        [183, 182, 144],
        ...,
        [223, 220, 215],
        [223, 220, 215],
        [223, 220, 215]],

       ...,

       [[ 90,  87,  77],
        [ 90,  87,  77],
        [ 91,  88,  78],
        ...,
        [106, 102,  96],
        [106, 102,  96],
        [106, 102,  96]],

       [[ 90,  87,  77],
        [ 91,  88,  78],
        [ 91,  88,  78],
        ...,
        [105, 101,  95],
        [105, 101,  95],
        [105, 101,  95]],

       [[ 90,  87,  77],
        [ 91,  88,  78],
        [ 91,  88,  78],
        ...,
        [104, 100,  94],
        [104, 100,  94],
        [104, 100,  94]]

In [10]:
image.shape

(500, 600, 3)

In [11]:
(h, w) = image.shape[:2]

(h, w)

(500, 600)

In [12]:
blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))
# 1.0 is the scale factor. we are not going to scale it

In [13]:
blob

array([[[[ 81.,  81.,  76., ..., 119., 119., 119.],
         [ 78.,  79.,  81., ..., 119., 119., 119.],
         [ 68.,  70.,  75., ..., 118., 118., 118.],
         ...,
         [-13., -13., -11., ...,  -1.,   1.,   2.],
         [-14., -13., -15., ...,   0.,   2.,   2.],
         [-14., -13., -14., ...,  -2.,   0.,   0.]],

        [[  6.,   7.,   1., ...,  43.,  43.,  43.],
         [  4.,   5.,   6., ...,  43.,  43.,  43.],
         [ -5.,  -2.,   0., ...,  42.,  42.,  42.],
         ...,
         [-89., -89., -86., ..., -76., -76., -74.],
         [-90., -89., -90., ..., -74., -75., -75.],
         [-90., -89., -90., ..., -76., -77., -77.]],

        [[ 22.,  21.,  13., ...,  92.,  92.,  92.],
         [ 23.,  21.,  20., ...,  92.,  92.,  92.],
         [ 18.,  19.,  16., ...,  91.,  91.,  91.],
         ...,
         [-45., -45., -42., ..., -33., -28., -26.],
         [-46., -45., -46., ..., -33., -27., -27.],
         [-46., -45., -45., ..., -35., -29., -29.]]]], dtype=float32)

In [14]:
blob.shape

(1, 3, 300, 300)

In [15]:
net.setInput(blob)
detections = net.forward()

In [16]:
detections    # this is the arry of the face

array([[[[0.        , 1.        , 0.9984427 , ..., 0.12488028,
          0.6709176 , 0.3542412 ],
         [0.        , 1.        , 0.12920395, ..., 3.9990287 ,
          4.8382664 , 4.984081  ],
         [0.        , 1.        , 0.12446587, ..., 3.994579  ,
          0.85450625, 4.978922  ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [17]:
detections.shape

(1, 1, 200, 7)

In [19]:
# loop over the detections

for i in range(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    
    if confidence > 0.5:
        # we need the x and y co-ordinates
        box = detections[0, 0, i, 3:7]*np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype('int')
        
        # we need to ensure that the bounding boxes fall within the dimensions of the frame
        (startX, startY) = (max(0, startX), max(0, startY))
        (endX, endY) = (min(w-1, endX), min(h-1, endY))
        
        # extract the face ROI, convert it from BGR to RGB channel
        # resize it to (224, 224) and preprocess it
        
        face = image[startY:endY, startX:endX]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis = 0)
        
        (mask, withoutMask) = model.predict(face)[0]
        
        # determine the class label and color we will use to draw the bounding box and text
        label = 'Mask' if mask > withoutMask else 'No Mask'
        color = (0, 255, 0) if label == 'Mask' else (0, 0, 255)
        
        # lets include teh probability of teh label
        label = '{} : {:.2f}%'.format(label, max(mask, withoutMask)*100)
        
        #display the label and the bounding boxes
        cv2.putText(image, label, (startX, startY-10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
        
cv2.imshow('OutPut', image)
cv2.waitKey(0)
cv2.destroyAllWindows()